In [2]:
from pathlib import Path
import numpy as np
import re
from math import prod
from collections import defaultdict, deque
from copy import copy
from itertools import pairwise, permutations, product

In [3]:
data = Path('../Data/Day21.txt').read_text().splitlines()

In [4]:
numpad = {
    '7': (0, 0),
    '8': (0, 1),
    '9': (0, 2),
    '4': (1, 0),
    '5': (1, 1),
    '6': (1, 2),
    '1': (2, 0),
    '2': (2, 1),
    '3': (2, 2),
    '0': (3, 1),
    'A': (3, 2)
}

dirpad = {
    '^': (0, 1),
    'A': (0, 2),
    '<': (1, 0),
    'v': (1, 1),
    '>': (1, 2)
}



def get_sequences(pad, forbidden):

    dr = {
        '^': (-1, 0),
        '>': (0, 1),
        'v': (1, 0),
        '<': (0, -1)
    }
    
    moves = {}

    for num, (i, j) in pad.items():
        for num2, (i2, j2) in pad.items():

            di = i2 - i
            dj = j2 - j

            up = '^'*abs(di)*(di < 0)
            down = 'v'*abs(di)*(di > 0)

            right = '>'*abs(dj)*(dj > 0)
            left = '<'*abs(dj)*(dj < 0)

            # Better to keep directions together
            # rather than mix
            # E.g. >>^^ and ^^>> are fine
            # But >^>^ is not
            sequence = [up, down, right, left]

            moves[num, num2] = set()

            for p in permutations(sequence):
                p = ''.join(p)
                if p in moves[num, num2]:
                    continue

                valid = True

                i3 = i
                j3 = j

                for move in p:
                    
                    di2, dj2 = dr[move]

                    i3 += di2
                    j3 += dj2

                    if (i3, j3) == forbidden:
                        valid = False
                        break
                if valid:
                    moves[num, num2].add(''.join(p))
    
    return moves
            

In [5]:
num_to_num = get_sequences(numpad, (3, 0))
dir_to_dir = get_sequences(dirpad, (0, 0))

In [12]:
def numcode_to_dircodes(code, num_to_num = num_to_num):
    return {'A'.join(p)+'A' for p in product(*[num_to_num[num1, num2] for num1, num2 in pairwise('A'+code)])}

cache = {}

def dircode_to_dircodes(code, dir_to_dir = dir_to_dir, cache = cache):

    if code in cache:
        return cache[code]

    if code.index('A') == len(code)-1:
        cache[code] = {'A'.join(p)+'A' for p in product(*[dir_to_dir[d1, d2] for d1, d2 in pairwise('A'+code)])}
        return cache[code]
    
    return {''.join(p) for p in product(*[dircode_to_dircodes(seg + 'A') for seg in code.split('A')[:-1]])}

def full_encode(code, drobots = 2):
    codes = numcode_to_dircodes(code)

    for _ in range(drobots):

        codes2 = set()

        for code in codes:
            codes2.update(dircode_to_dircodes(code))
        
        codes = codes2

    return codes

In [13]:
numcode_to_dircodes('029A')

{'<A^A>^^AvvvA', '<A^A^^>AvvvA'}

In [15]:
len(full_encode('029A', 2))

768

In [64]:
{''.join(p) for p in product(*[dircode_to_dircodes(code + 'A') for code in '^^<AAAA>>^A'.split('A')[:-1]])}

{'<AAv<A>>^AAAAvAA<^A>A', '<AAv<A>>^AAAAvAA^<A>A'}

In [53]:
'<AAv<A>>^AvAA<^A'.split('A')

['<', '', 'v<', '>>^', 'v', '', '<^', '']

In [57]:
'<^A'.index('A')

2

In [70]:
d2d('<AAv<A>>^AvAA<^A')

{'v<<A>>^AA<vA<A>>^AvAA<^A>A<vA>^AAv<<A>^A>A',
 'v<<A>>^AA<vA<A>>^AvAA<^A>A<vA^>AAv<<A>^A>A',
 'v<<A>>^AA<vA<A>>^AvAA<^A>Av<A>^AAv<<A>^A>A',
 'v<<A>>^AA<vA<A>>^AvAA<^A>Av<A^>AAv<<A>^A>A',
 'v<<A>>^AA<vA<A>>^AvAA^<A>A<vA>^AAv<<A>^A>A',
 'v<<A>>^AA<vA<A>>^AvAA^<A>A<vA^>AAv<<A>^A>A',
 'v<<A>>^AA<vA<A>>^AvAA^<A>Av<A>^AAv<<A>^A>A',
 'v<<A>>^AA<vA<A>>^AvAA^<A>Av<A^>AAv<<A>^A>A',
 'v<<A>>^AAv<A<A>>^AvAA<^A>A<vA>^AAv<<A>^A>A',
 'v<<A>>^AAv<A<A>>^AvAA<^A>A<vA^>AAv<<A>^A>A',
 'v<<A>>^AAv<A<A>>^AvAA<^A>Av<A>^AAv<<A>^A>A',
 'v<<A>>^AAv<A<A>>^AvAA<^A>Av<A^>AAv<<A>^A>A',
 'v<<A>>^AAv<A<A>>^AvAA^<A>A<vA>^AAv<<A>^A>A',
 'v<<A>>^AAv<A<A>>^AvAA^<A>A<vA^>AAv<<A>^A>A',
 'v<<A>>^AAv<A<A>>^AvAA^<A>Av<A>^AAv<<A>^A>A',
 'v<<A>>^AAv<A<A>>^AvAA^<A>Av<A^>AAv<<A>^A>A'}

In [68]:
dircode_to_dircodes('<AAv<A>>^AvAA<^A')

{'v<<A>>^AA<vA<A>>^AvAA<^A>A<vA>^AAv<<A>^A>A',
 'v<<A>>^AA<vA<A>>^AvAA<^A>A<vA^>AAv<<A>^A>A',
 'v<<A>>^AA<vA<A>>^AvAA<^A>Av<A>^AAv<<A>^A>A',
 'v<<A>>^AA<vA<A>>^AvAA<^A>Av<A^>AAv<<A>^A>A',
 'v<<A>>^AA<vA<A>>^AvAA^<A>A<vA>^AAv<<A>^A>A',
 'v<<A>>^AA<vA<A>>^AvAA^<A>A<vA^>AAv<<A>^A>A',
 'v<<A>>^AA<vA<A>>^AvAA^<A>Av<A>^AAv<<A>^A>A',
 'v<<A>>^AA<vA<A>>^AvAA^<A>Av<A^>AAv<<A>^A>A',
 'v<<A>>^AAv<A<A>>^AvAA<^A>A<vA>^AAv<<A>^A>A',
 'v<<A>>^AAv<A<A>>^AvAA<^A>A<vA^>AAv<<A>^A>A',
 'v<<A>>^AAv<A<A>>^AvAA<^A>Av<A>^AAv<<A>^A>A',
 'v<<A>>^AAv<A<A>>^AvAA<^A>Av<A^>AAv<<A>^A>A',
 'v<<A>>^AAv<A<A>>^AvAA^<A>A<vA>^AAv<<A>^A>A',
 'v<<A>>^AAv<A<A>>^AvAA^<A>A<vA^>AAv<<A>^A>A',
 'v<<A>>^AAv<A<A>>^AvAA^<A>Av<A>^AAv<<A>^A>A',
 'v<<A>>^AAv<A<A>>^AvAA^<A>Av<A^>AAv<<A>^A>A'}

In [46]:
cache = {'A': 'A'}

def d2d(code):
    

{'v<<A>>^AA'}

In [47]:
dircode_to_dircodes('v<A')

{'<vA<A>>^A', 'v<A<A>>^A'}

In [40]:
dircode_to_dircodes('>>^')

{'vAA<^A', 'vAA^<A'}

In [156]:
for k, v in num_to_num.items():
    for moves in v:
        print(k, v, moves, dircode_to_dircodes(moves))

('7', '7') {''}  {'A'}
('7', '8') {'>'} > {'vA'}
('7', '9') {'>>'} >> {'vAA'}
('7', '4') {'v'} v {'v<A', '<vA'}
('7', '5') {'>v', 'v>'} >v {'vA<A'}
('7', '5') {'>v', 'v>'} v> {'v<A>A', '<vA>A'}
('7', '6') {'>>v', 'v>>'} >>v {'vAA<A'}
('7', '6') {'>>v', 'v>>'} v>> {'<vA>AA', 'v<A>AA'}
('7', '1') {'vv'} vv {'v<AA', '<vAA'}
('7', '2') {'vv>', '>vv'} vv> {'<vAA>A', 'v<AA>A'}
('7', '2') {'vv>', '>vv'} >vv {'vA<AA'}
('7', '3') {'>>vv', 'vv>>'} >>vv {'vAA<AA'}
('7', '3') {'>>vv', 'vv>>'} vv>> {'v<AA>AA', '<vAA>AA'}
('7', '0') {'>vvv'} >vvv {'vA<AAA'}
('7', 'A') {'>>vvv'} >>vvv {'vAA<AAA'}
('8', '7') {'<'} < {'v<<A'}
('8', '8') {''}  {'A'}
('8', '9') {'>'} > {'vA'}
('8', '4') {'v<', '<v'} v< {'v<A<A', '<vA<A'}
('8', '4') {'v<', '<v'} <v {'v<<A>A'}
('8', '5') {'v'} v {'v<A', '<vA'}
('8', '6') {'>v', 'v>'} >v {'vA<A'}
('8', '6') {'>v', 'v>'} v> {'v<A>A', '<vA>A'}
('8', '1') {'<vv', 'vv<'} <vv {'v<<A>AA'}
('8', '1') {'<vv', 'vv<'} vv< {'<vAA<A', 'v<AA<A'}
('8', '2') {'vv'} vv {'v<AA', '<vAA'}
('8

In [166]:
for k, v in dir_to_dir.items():
    for moves in v:
        print(k, v, moves, dircode_to_dircodes(moves))

('^', '^') {''}  {'A'}
('^', 'A') {'>'} > {'vA'}
('^', '<') {'v<'} v< {'v<A<A', '<vA<A'}
('^', 'v') {'v'} v {'v<A', '<vA'}
('^', '>') {'>v', 'v>'} >v {'vA<A'}
('^', '>') {'>v', 'v>'} v> {'v<A>A', '<vA>A'}
('A', '^') {'<'} < {'v<<A'}
('A', 'A') {''}  {'A'}
('A', '<') {'v<<'} v<< {'v<A<AA', '<vA<AA'}
('A', 'v') {'v<', '<v'} v< {'v<A<A', '<vA<A'}
('A', 'v') {'v<', '<v'} <v {'v<<A>A'}
('A', '>') {'v'} v {'v<A', '<vA'}
('<', '^') {'>^'} >^ {'vA<^A', 'vA^<A'}
('<', 'A') {'>>^'} >>^ {'vAA<^A', 'vAA^<A'}
('<', '<') {''}  {'A'}
('<', 'v') {'>'} > {'vA'}
('<', '>') {'>>'} >> {'vAA'}
('v', '^') {'^'} ^ {'<A'}
('v', 'A') {'>^', '^>'} >^ {'vA<^A', 'vA^<A'}
('v', 'A') {'>^', '^>'} ^> {'<Av>A', '<A>vA'}
('v', '<') {'<'} < {'v<<A'}
('v', 'v') {''}  {'A'}
('v', '>') {'>'} > {'vA'}
('>', '^') {'^<', '<^'} ^< {'<Av<A'}
('>', '^') {'^<', '<^'} <^ {'v<<A>^A'}
('>', 'A') {'^'} ^ {'<A'}
('>', '<') {'<<'} << {'v<<AA'}
('>', 'v') {'<'} < {'v<<A'}
('>', '>') {''}  {'A'}


In [12]:
full_encode('0290', 3)

{'<vA<A>>^Av<<A>>^AAvAA^<A>Av<A>^AA<Av<A>>^AvA^Av<A<AA>>^AvAA<^A>A<vA^>A<A>Av<A<A>>^A<Av>A^A<vA<AA>>^AvA^<A>AvA^AAv<A>^A<A>A<vA<A>>^Av<<A>>^AvAA<^A>AAA<vA<AA>>^AvAA<^A>Av<A>^AA<Av<A>>^AvA^A',
 'v<A<A>>^Av<<A>>^AAvAA<^A>Av<A>^AAv<<A>^A>AvA^A<vA<AA>>^AvAA<^A>Av<A^>A<A>A<vA<A>>^A<Av>A^Av<A<AA>>^AvA<^A>AvA^AA<vA>^A<A>A<vA<A>>^Av<<A>>^AvAA<^A>AAA<vA<AA>>^AvAA^<A>A<vA^>AAv<<A>^A>AvA^A',
 'v<<A>A^>Av<<A>>^AAvAA<^A>Av<A>^AAv<<A>^A>AvA^A<vA<AA>>^AvAA<^A>A<vA>^A<A>A<vA<AA>>^AvAA^<A>AAv<A>^Av<<A>>^A<A>vA^Av<<A>>^AvA^Av<<A>A^>Av<<A>>^AAvAA<^A>Av<A>^A<A>AAA<vA>^Av<<A>^A>AvA^A',
 'v<<A>A>^Av<<A>>^AAvAA<^A>A<vA^>AAv<<A>^A>AvA^Av<A<AA>>^AvAA<^A>A<vA>^A<A>Av<A<A>>^AvA<^A>A<vA<AA>>^AvA<^A>AvA^AAv<A>^A<A>A<vA<A>>^Av<<A>>^AvAA^<A>AAAv<A<AA>>^AvAA<^A>A<vA>^AAv<<A>^A>AvA^A',
 'v<<A>A>^Av<<A>>^AAvAA^<A>A<vA>^AA<Av<A>>^AvA^Av<A<AA>>^AvAA^<A>Av<A>^A<A>Av<<A>A^>A<A>vA^Av<<A>>^Av<A<A>>^AvAA<^A>AAv<A>^A<A>A<vA<A>>^Av<<A>>^AAvAA^<A>A<vA>^A<A>AAAv<A>^A<Av<A>>^AvA^A',
 'v<A<A>>^Av<<A>>^AAvAA<^A>Av<A^>AA<Av<A>>^AvA^A